# Neuronale Netze

In dieser Übung schauen wir uns zwei sehr einfache Beispiele für neuronale Netze an. Wenn ihr darüber hinausgehend Interesse an neuronalen Netzen habt, empfehle ich die Lehrveranstaltung "AI 3: Artificial Neural Networks" im Sommersemester.

Falls ihr an der Theorie und den mathematischen Details interessiert seid, ist aus meiner Sicht folgendes Buch sehr empfehlenswert: https://www.deeplearningbook.org/ (u.a. von Yoshua Bengio, der zusammen mit Geoffrey Hinton und Yann LeCun den Turing-Award 2018 für Arbeiten an Deep Learning-Methoden erhalten hat).

Es gibt natürlich eine vielzahl weiterführender praktischer Tutorials für Deep Learning. Aber Achtung: Viele Tutorials zeigen nur sehr knapp, wie man ein neuronales Netz trainiert und evaluiert, ignorieren aber das oft etwas mühsame, aber wichtige Drumherum, also das Vorbereiten des Datensatzes usw. Dieses Tutorial zeigt das hingegen sehr schön: https://www.pyimagesearch.com/2018/09/10/keras-tutorial-how-to-get-started-with-keras-deep-learning-and-python/

*Aufgabe*: Erläutere die Kernidee und den Aufbau von Neuronen und Neuronalen Netzen!

In [ ]:
import pandas as pd

df = pd.read_csv("iris.csv", index_col=0)
df

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df.SepalWidth.values, df.PetalLength.values,'o')

## Neuronale Netze für Regression



In dieser Übung verwenden wir `keras` für das Training von neuronalen Netzen. Weitere Informationen zu diesem Paket finden Sie hier: https://keras.io/

Die Beispiele, die wir uns in der Übung anschauen, sollten aber (hoffentlich) recht selbsterklärend sein. In dem Beispiel unten beschreiben wir die Netzwerkstruktur in Keras. Dieses Netzwerk besteht nur aus einem einzigen Knoten, die Aktivierungsfunktion ist die Identität. Zudem legen wir fest, dass wir den Mean Squared Error als Loss-Funktion nutzen wollen, und den Adam-Optimizer. 

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, Softmax
import matplotlib.pyplot as plt

model = Sequential([ 
    Dense(1,                         # number of units in the layer
          input_shape=(1,),          # shape of the inputs
          activation='linear',       # the unit computes a simple linear (identity function) on its input
          kernel_initializer='ones', # the weights (there is just one in this example) are initialised to 1
          use_bias=True             # the unit does not use a bias term as offset
)])

model.compile(optimizer='adam', loss='mse')


Nun trainieren und evaluieren wir das Modell. Das Array `x` wird später für den Plot der Netzwerkfunktion benötigt. Dann trainieren wir ein Modell mit SepalWidth als Input und PetalLength als Output. Alle 25 Trainingsepochen bestimmen wir die Prädiktion der Werte in `x` mit der Funktion `predict` (diese benötigen wir, um die Netzwerkfunktion zu plotten) und geben den Wert der Loss-Funktion aus. 

In [ ]:
x = np.linspace(0,5,num=100)
# let's train the network for 5 iterations of 25 epochs each ...
iterations = 5
epochs_per_iter = 25
number_of_ephochs_trained=0
for i in range(iterations):
    model.fit(df.SepalWidth, df.PetalLength, epochs=epochs_per_iter, verbose=0)
    number_of_ephochs_trained = number_of_ephochs_trained + epochs_per_iter
    y = model.predict(x)
    l = model.evaluate(df.SepalWidth, df.PetalLength, verbose=0)
    print('Loss after %i epochs: %f' % (number_of_ephochs_trained, l))
    plt.plot(x,y, label="%i epochs" % (number_of_ephochs_trained))

plt.legend()
plt.xticks(np.arange(0, 5, 1))
plt.yticks(np.arange(0, 7, 1))

def show(plt, title=None, xlabel=None, ylabel=None, legend=True, grid=True):
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    if grid: plt.grid()
    if legend: plt.legend()
    plt.show()
    
def get_coefficient(m):
    return model.layers[0].get_weights()[0][0][0]

def show_coefficient(m):
    print("The network's coefficient is: %5f" % get_coefficient(m))

    
show_coefficient(model)

show(plt, 'Network function for different epochs', 'x', 'y')

*Aufgabe*: Was für eine Funktion stellt dieses Netzwerk dar?

*Aufgabe*: Was passiert, wenn der Input von der Form $x = [x_1, x_2, ..., x_n]^T$ ist? Wie sieht unsere Gewichtsmatrix $W$ aus? Wie lautet die Gleichung für $\hat{y}$?
* $W =$ ?
* $\hat{y} =$ ?

*Aufgabe*: Was passiert, wenn wir zusätzlich unser Layer auf k Units/Neuronen erweitern? Wie sieht unsere Gewichtsmatrix $W$ aus? Wie lautet die Gleichung für $\mathbf{\hat{y}}$?
* $W =$ ?
* $\mathbf{\hat{y}} =$ ?

*Aufgabe*: Erklären sie, wie ein neuronales Netz lernt! Erläutern Sie in diesem Zusammenhang alle mathematisch notwendigen Konzepte (Kettenregel, Jacobi-Matrix, Gradienten, Gradientenabstiegsverfahren usw.).

*Aufgabe*: Vergleichen Sie die Ergebnisse mit der linearen Regression von Übung 2. Fügen Sie weitere Neuronen hinzu und verändern Sie die Aktivierungsfunktion. Wie ändert sich die der Loss und die Netzwerkfunktion?

In [ ]:
model = Sequential([ 
    Dense(1,                         # number of units in the layer
          input_shape=(1,),          # shape of the inputs
          activation='linear')     # the unit computes a simple linear (identity function) on its input
])

model.compile(optimizer='adam', loss='mse')

x = np.linspace(0,5,num=100)
# let's train the network for 5 iterations of 25 epochs each ...
iterations = 5
epochs_per_iter = 100
number_of_ephochs_trained=0
for i in range(iterations):
    model.fit(df.SepalWidth.values, df.PetalLength.values, epochs=epochs_per_iter, verbose=0)
    number_of_ephochs_trained = number_of_ephochs_trained + epochs_per_iter
    y = model.predict(x)
    l = model.evaluate(df.SepalWidth.values, df.PetalLength.values, verbose=0)
    print('Loss after %i epochs: %f' % (number_of_ephochs_trained, l))
    plt.plot(x,y, label="%i epochs" % (number_of_ephochs_trained))

plt.legend()
plt.xticks(np.arange(0, 5, 1))
plt.yticks(np.arange(0, 7, 1))

def show(plt, title=None, xlabel=None, ylabel=None, legend=True, grid=True):
    if title: plt.title(title)
    if xlabel: plt.xlabel(xlabel)
    if ylabel: plt.ylabel(ylabel)
    if grid: plt.grid()
    if legend: plt.legend()
    plt.show()
    

#print(model.get_weights())
    
show(plt, 'Network function for different epochs', 'x', 'y')

## MNIST
Als nächstes schauen wir uns einen deutlich komplizierteren Datensatz an: MNIST enthält Bilder der Größe 28*28 von handgeschriebenen Ziffern. Die Aufgabe ist es, einen Klassifikator für diese Ziffern zu entwickeln. 

Zunächst laden wir den Datensatz und konvertieren die Klassen-Labels zu Vektoren mit One-Hot-Encoding.

In [ ]:
from keras.utils import to_categorical
import numpy as np
import pandas as pd
import keras
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Input, Flatten, Softmax
import matplotlib.pyplot as plt
import tensorflow as tf

dataset = keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = dataset.load_data()

# reshape each image into a vector of length 28*28=784
train_images = train_images.reshape((60000,28*28))
test_images = test_images.reshape((10000,28*28))

train_images, test_images = train_images / 255.0, test_images / 255.0
# turn labels into categories
train_labels_c = to_categorical(train_labels)
test_labels_c  = to_categorical(test_labels)



Dann erstellen wir ein sehr einfaches Netzwerk mit 10 Neuronen und wählen wieder mean squared error als Loss-Funktion (ist das in diesem Fall sinnvoll?)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10),
])

In [ ]:
model.compile(optimizer='adam',
               loss="mse"
              ,metrics=['accuracy'])

Schließlich trainieren wir das Modell und erreichen eine Validation Accuracy von ca. 84%. 

In [ ]:
h = model.fit(train_images, train_labels_c, validation_data=(test_images, test_labels_c), verbose=1)

### Aufgabe
Verbessern Sie das Modell, indem Sie geeignetere Parameter (z.B. Netzwerkarchitektur, Loss-Funktion) wählen. Welche Validation Accuracy erreicht Ihr Modell?